In [7]:
import numpy as np 
import urdf2casadi.urdfparser as u2c
from urdf2casadi.geometry import plucker
from urdf_parser_py.urdf import URDF, Pose
import PyKDL as kdl
import kdl_parser_py.urdf as kdlurdf
from timeit import Timer, timeit, repeat
import rbdl
import pybullet as pb

In [8]:
def median(lst):
    n = len(lst)
    if n < 1:
            return None
    if n % 2 == 1:
            return sorted(lst)[n//2]
    else:
            return sum(sorted(lst)[n//2-1:n//2+1])/2.0
        
def average(lst): 
    return sum(lst) / len(lst) 


In [29]:
def id_u2c_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    id_u2c(q, qdot, qddot)
    
def id_pb_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    
    id_pb = pb.calculateInverseDynamics(pbmodel, q, qdot, qddot)
    
def id_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.InverseDynamics(rbdlmodel, q_np, qdot_np, qddot_np, id_rbdl)
    

    
    
def C_u2c_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    C_u2c(q, qdot)
    
def C_pb_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    C_pb = pb.calculateInverseDynamics(pbmodel, q, qdot, zeros_pb)
    
def C_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.InverseDynamics(rbdlmodel, q_np, qdot_np, zeros_rbdl, id_rbdl)

def C_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToCoriolis(q_kdl, qdot_kdl, C_kdl)
    
    
    

def M_u2c_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    M_u2c(q)
    
def M_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToMass(q_kdl, M_kdl)

def M_pb_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    M_pb = pb.calculateMassMatrix(pbmodel, q)
    
def M_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.CompositeRigidBodyAlgorithm(rbdlmodel, q_np, M_rbdl)

    


    
def g_u2c_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g_u2c(q)
    
def g_pb_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    g_pb = pb.calculateInverseDynamics(pbmodel, q, zeros_pb, zeros_pb)
    
def g_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.InverseDynamics(rbdlmodel, q_np, zeros_rbdl, zeros_rbdl, id_rbdl)

def g_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToGravity(q_kdl, g_kdl)
    
    
    
    

def fd_aba_u2c_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_aba_u2c(q, qdot, tau)
    

def fd_crba_u2c_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_crba_u2c(q, qdot, tau)
    
def fd_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.ForwardDynamics(rbdlmodel, q_np, qdot_np, tau_np, qddot_np)

In [23]:
sim = pb.connect(pb.DIRECT)

#storage for timing data
mediantime_id_u2c = 0
mediantime_M_u2c = 0
mediantime_C_u2c = 0
mediantime_g_u2c = 0
mediantime_fd_crba_u2c = 0
mediantime_fd_aba_u2c = 0


mediantime_id_rbdl = 0
mediantime_g_rbdl = 0
mediantime_C_rbdl = 0
mediantime_fd_aba_rbdl = 0

mediantime_M_kdl = 0
mediantime_g_kdl = 0
mediantime_C_kdl = 0

mediantime_id_pb = 0
mediantime_g_pb = 0
mediantime_C_pb = 0
mediantime_M_pb = 0

nitr = 1
nrepeat = 10000

path_to_urdf = "/home/lmjohann/urdf2casadi/examples/urdf/kukalwr.urdf"
root = 'calib_kuka_arm_base_link'
tip = "kuka_arm_7_link"

ok, kdl_tree = kdlurdf.treeFromFile(path_to_urdf)
kdlmodel = kdl_tree.getChain(root,tip)

rbdlmodel = rbdl.loadModel(path_to_urdf)
pbmodel = pb.loadURDF(path_to_urdf, useFixedBase=True, flags = pb.URDF_USE_INERTIA_FROM_FILE)
robot = u2c.URDFparser()
robot_desc = robot.from_file(path_to_urdf)

jlist, names, qmax, qmin = robot.get_joint_info(root, tip)
njoints = len(jlist)

gravity_kdl = kdl.Vector()
gravity_kdl[2] = -9.81
gravity_u2c = [0, 0, -9.81]

#rbdl declarations
q_np = np.zeros(njoints)
qdot_np = np.zeros(njoints)
qddot_np = np.zeros(njoints)
tau_np = np.zeros(njoints)
g_rbdl = np.zeros(njoints)
id_rbdl = np.zeros(njoints)
C_rbdl = np.zeros(njoints)
zeros_rbdl = np.zeros(njoints)


#kdl declatations
q_kdl = kdl.JntArray(njoints)
qdot_kdl = kdl.JntArray(njoints)
g_kdl = kdl.JntArray(njoints)
C_kdl = kdl.JntArray(njoints)

#u2c and pybullet declarations
q = [None]*njoints
qdot = [None]*njoints
qddot = [None]*njoints
tau = [None]*njoints
g_pb = [None]*njoints
M_pb = [None]*njoints
id_pb = [None]*njoints

zeros_pb = []
for i in range(njoints):
    zeros_pb.append(0.)

                  
M_kdl = kdl.JntSpaceInertiaMatrix(njoints)
M_rbdl = (rbdlmodel.q_size, rbdlmodel.q_size)
M_rbdl = np.zeros(M_rbdl)

gravity_u2c = [0, 0, -9.81]
C_u2c = robot.get_coriolis_rnea(root, tip)
g_u2c = robot.get_gravity_rnea(root, tip, gravity_u2c)
M_u2c = robot.get_inertia_matrix_crba(root, tip)
id_u2c = robot.get_inverse_dynamics_rnea(root, tip, gravity_u2c)
fd_aba_u2c = robot.get_forward_dynamics_aba(root, tip, gravity_u2c)
fd_crba_u2c = robot.get_forward_dynamics_crba(root, tip, gravity_u2c)

The root link calib_kuka_arm_base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
/home/lmjohann/urdf2casadi/examples/urdf/kukalwr.urdf


In [24]:
timeit_g_u2c = repeat("g_u2c_func()", setup = "from __main__ import g_u2c_func", repeat = nrepeat, number = nitr)
mediantime_g_u2c = median(timeit_g_u2c)

timeit_C_u2c = repeat("C_u2c_func()", setup = "from __main__ import C_u2c_func", repeat = nrepeat, number = nitr)
mediantime_C_u2c = median(timeit_C_u2c)

timeit_M_u2c = repeat("M_u2c_func()", setup = "from __main__ import M_u2c_func", repeat = nrepeat, number = nitr)
mediantime_M_u2c = median(timeit_M_u2c)

timeit_id_u2c = repeat("id_u2c_func()", setup = "from __main__ import id_u2c_func", repeat = nrepeat, number = nitr)
mediantime_id_u2c = median(timeit_id_u2c)

timeit_fd_crba_u2c = repeat("fd_crba_u2c_func()", setup = "from __main__ import fd_crba_u2c_func", repeat = nrepeat, number = nitr)
mediantime_fd_crba_u2c = median(timeit_fd_crba_u2c)

timeit_fd_aba_u2c = repeat("fd_aba_u2c_func()", setup = "from __main__ import fd_aba_u2c_func", repeat = nrepeat, number = nitr)
mediantime_fd_aba_u2c = median(timeit_fd_aba_u2c)


print "\ntime g u2c:\n", mediantime_g_u2c
print "\ntime id u2c:\n", mediantime_id_u2c
print "\ntime C u2c:\n", mediantime_C_u2c
print "\ntime M u2c:\n", mediantime_M_u2c
print "\ntime fd aba u2c:\n", mediantime_fd_aba_u2c
print "\ntime fd crba u2c:\n", mediantime_fd_crba_u2c


time g u2c:
1.78813934326e-05

time id u2c:
4.10079956055e-05

time C u2c:
2.90870666504e-05

time M u2c:
1.78813934326e-05

time fd aba u2c:
4.10079956055e-05

time fd crba u2c:
4.29153442383e-05


In [25]:
timeit_g_pb = repeat("g_pb_func()", setup = "from __main__ import g_pb_func", repeat = nrepeat, number = nitr)
mediantime_g_pb = median(timeit_g_pb)

timeit_C_pb = repeat("C_pb_func()", setup = "from __main__ import C_pb_func", repeat = nrepeat, number = nitr)
mediantime_C_pb = median(timeit_C_pb)

timeit_id_pb = repeat("id_pb_func()", setup = "from __main__ import id_pb_func", repeat = nrepeat, number = nitr)
mediantime_id_pb = median(timeit_id_pb)

timeit_M_pb = repeat("M_pb_func()", setup = "from __main__ import M_pb_func", repeat = nrepeat, number = nitr)
mediantime_M_pb = median(timeit_M_pb)

print "\ntime g pb:\n", mediantime_g_pb
print "\ntime id pb:\n", mediantime_id_pb
print "\ntime C pb:\n", mediantime_C_pb
print "\ntime M pb:\n", mediantime_M_pb


time g pb:
5.96046447754e-06

time id pb:
1.28746032715e-05

time C pb:
1.00135803223e-05

time M pb:
6.91413879395e-06


In [30]:
timeit_g_kdl = repeat("g_kdl_func()", setup = "from __main__ import g_kdl_func", repeat = nrepeat, number = nitr)
mediantime_g_kdl = median(timeit_g_kdl)

timeit_C_kdl = repeat("C_kdl_func()", setup = "from __main__ import C_kdl_func", repeat = nrepeat, number = nitr)
mediantime_C_kdl = median(timeit_C_kdl)

timeit_M_kdl = repeat("M_kdl_func()", setup = "from __main__ import M_kdl_func", repeat = nrepeat, number = nitr)
mediantime_M_kdl = median(timeit_M_kdl)

print "\ntime g kdl:\n", mediantime_g_kdl
print "\ntime C kdl:\n", mediantime_C_kdl
print "\ntime M kdl:\n", mediantime_M_kdl


time g kdl:
1.50203704834e-05

time C kdl:
1.90734863281e-05

time M kdl:
1.59740447998e-05


In [28]:
timeit_g_rbdl = repeat("g_rbdl_func()", setup = "from __main__ import g_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_g_rbdl = median(timeit_g_rbdl)

timeit_C_rbdl = repeat("C_rbdl_func()", setup = "from __main__ import C_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_C_rbdl = median(timeit_C_rbdl)

timeit_id_rbdl = repeat("id_rbdl_func()", setup = "from __main__ import id_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_id_rbdl = median(timeit_id_rbdl)

timeit_M_rbdl = repeat("M_rbdl_func()", setup = "from __main__ import M_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_M_rbdl = median(timeit_M_rbdl)

timeit_fd_rbdl = repeat("fd_rbdl_func()", setup = "from __main__ import fd_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_fd_rbdl = median(timeit_fd_rbdl)

print "\ntime g rbdl:\n", mediantime_g_rbdl
print "\ntime id rbdl:\n", mediantime_id_rbdl
print "\ntime C rbdl:\n", mediantime_C_rbdl
print "\ntime M rbdl:\n", mediantime_M_rbdl
print "\ntime fd rbdl:\n", mediantime_fd_rbdl



time g rbdl:
8.10623168945e-06

time id rbdl:
1.50203704834e-05

time C rbdl:
1.19209289551e-05

time M rbdl:
6.91413879395e-06

time fd rbdl:
1.69277191162e-05


In [31]:
print timeit_g_u2c

[0.00021791458129882812, 0.0002181529998779297, 8.392333984375e-05, 2.288818359375e-05, 2.193450927734375e-05, 2.09808349609375e-05, 3.0040740966796875e-05, 3.719329833984375e-05, 4.315376281738281e-05, 3.218650817871094e-05, 2.09808349609375e-05, 2.002716064453125e-05, 2.002716064453125e-05, 2.1219253540039062e-05, 2.002716064453125e-05, 2.002716064453125e-05, 2.002716064453125e-05, 2.09808349609375e-05, 2.7894973754882812e-05, 2.002716064453125e-05, 3.3855438232421875e-05, 3.2901763916015625e-05, 3.409385681152344e-05, 3.600120544433594e-05, 3.314018249511719e-05, 3.314018249511719e-05, 3.2901763916015625e-05, 3.1948089599609375e-05, 2.8848648071289062e-05, 2.09808349609375e-05, 2.002716064453125e-05, 2.09808349609375e-05, 1.9073486328125e-05, 1.9073486328125e-05, 2.002716064453125e-05, 1.9073486328125e-05, 2.002716064453125e-05, 2.002716064453125e-05, 1.9073486328125e-05, 2.09808349609375e-05, 2.002716064453125e-05, 2.002716064453125e-05, 2.09808349609375e-05, 1.9073486328125e-05, 1